In [6]:
!pip install --upgrade pip
!pip install transformers accelerate gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 20.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [7]:
import torch
print("GPU available:", torch.cuda.is_available())

GPU available: True


In [ ]:
from huggingface_hub import login
login(token="")    #own huggingface token

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

MODEL_NAME = "gpt2"   # change to any model repo you want

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype="auto",
    trust_remote_code=True
)

text_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    temperature=0.7,
    top_p=0.9
)

result = text_pipe("Hello! Can you explain what AI is?", do_sample=True)
print(result[0]["generated_text"])


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello! Can you explain what AI is?

AI is an artificial intelligence. It is an intelligence that has no information about its environment, or about its environment itself. It does not know how to interact with objects. It does not know how to react to things. It does not know how to behave in other situations. It does not know how to respond to stimuli. It does not know how to respond to other people.

AI is not a social system. It is a computer program that generates and communicates with a human. It has no special knowledge about human behavior. It has no special knowledge about the human body. It does not know how to interact with other


In [10]:
!pip install --upgrade googlesearch-python newspaper3k lxml_html_clean

from googlesearch import search
from newspaper import Article
import logging

def web_search(query, max_results=5, snippet_chars=300):
    """
    Run a Google search for `query`, return up to `max_results` results.
    Each result is parsed with newspaper3k.
    Returns: list of dicts [{title, href, body}]
    """
    results = []
    for url in search(query, num_results=max_results, lang="en"):
        try:
            art = Article(url)
            art.download()
            art.parse()
            text = art.text.replace("\n", " ").strip()
            snippet = text[:snippet_chars] + ("…" if len(text) > snippet_chars else "")
            results.append({
                "title": art.title or url,
                "href": url,
                "body": snippet
            })
        except Exception as e:
            logging.warning(f"Failed to parse {url}: {e}")
        if len(results) >= max_results:
            break
    return results
results = web_search("latest AI news", max_results=3)

for i, r in enumerate(results, start=1):
    print(f"\nResult {i}:")
    print("Title :", r["title"])
    print("Link  :", r["href"])
    print("Body  :", r["body"])



Result 1:
Title : Latest AI News, Analysis & Events
Link  : https://www.artificialintelligence-news.com/
Body  : Check out informative stories and events that are currently in our Spotlight!  Currently in the Spotlight – Artificial General Intelligence (AGI)

Result 2:
Title : AI News & Artificial Intelligence
Link  : https://techcrunch.com/category/artificial-intelligence/
Body  : Put your brand in front of 10,000+ tech and VC leaders across all three days of Disrupt 2025. Amplify your reach, spark real connections, and lead the innovation charge. Secure your exhibit space before your competitor does.


In [11]:
def answer_with_citations(user_query):
    hits = web_search(user_query, max_results=10)

    prompt_parts = [
        "You are an AI assistant. Use the following search results to answer the question.\n"
        "Explain clearly, list points if needed, and cite each source by its link.\n"
    ]

    for idx, hit in enumerate(hits, start=1):
        title = hit.get("title", "").strip()
        body  = hit.get("body", "").strip()
        url   = hit.get("href", hit.get("url", "")).strip()
        prompt_parts.append(f"[{idx}] {title}\n{body}\nURL: {url}\n")

    prompt_parts.append(f"Question: {user_query}\nAnswer:")
    prompt = "\n".join(prompt_parts)

    generated = llama_pipe(prompt, max_new_tokens=512, temperature=0.7)[0]["generated_text"]
    answer = generated[len(prompt):].strip()
    return answer


In [14]:
import gradio as gr

def chat_fn(user_message, history):
    history = history + [(user_message, None)]
    bot_reply = answer_with_citations(user_message)
    history[-1] = (user_message, bot_reply)
    return history, history, ""

css = """
.gradio-container { display: flex; justify-content: center; padding: 2rem; }
.chat-box { width: 100%; max-width: 700px; }
.chatbot .message.user {
  background-color: #007AFF;
  color: white;
  border-radius: 16px 16px 0 16px;
  padding: 8px 12px;
  margin: 4px 0;
  align-self: flex-end;
  max-width: 80%;
}
.chatbot .message.bot {
  background-color: #FFA500;
  color: #111;
  border-radius: 16px 16px 16px 0;
  padding: 8px 12px;
  margin: 4px 0;
  align-self: flex-start;
  max-width: 80%;
}
input[type="text"] {
  border-radius: 20px;
  padding: 10px 16px;
  border: 1px solid #CCC;
}
"""

with gr.Blocks(css=css, title="📡 LLaMA 3.2 3B Web-Search Agent") as demo:
    gr.Markdown("## Ask anything and get cited answers")
    with gr.Group(elem_classes="chat-box"):
        chatbot = gr.Chatbot(elem_classes="chatbot", height=500)
        state   = gr.State([])
        user_input = gr.Textbox(
            placeholder="Type your question and hit enter",
            show_label=False
        )
        clear_btn = gr.Button("🗑️ Clear Chat")

    user_input.submit(
        fn=chat_fn,
        inputs=[user_input, state],
        outputs=[chatbot, state, user_input]
    )

    clear_btn.click(
        fn=lambda: ([], [], ""),
        inputs=None,
        outputs=[chatbot, state, user_input]
    )

demo.launch(share=True)


/tmp/ipython-input-529351874.py:40: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(elem_classes="chatbot", height=500)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://e22cf080e01deaf971.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
